In [1]:
!pip install datasets
!pip install seacrowd accelerate peft bitsandbytes wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.9/744.9 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 93.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 4.

In [2]:
%%capture
!pip install unsloth vllm
!pip install triton==3.1.0
!pip install -U pynvml
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [3]:
import pandas as pd
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import random 
import torch
import datasets

from tqdm import tqdm


In [4]:
# Detect train, dev, and test files
DATASET_ROOT = '/kaggle/input/indosum/indosum'

files_id_dir = os.listdir(DATASET_ROOT)
train_files = []
dev_files = []
test_files = []

for filename in files_id_dir:
    if 'train' in filename:
        train_files.append(filename)
    elif 'dev' in filename:
        dev_files.append(filename)
    elif 'test' in filename:
        test_files.append(filename)

train_files, dev_files, test_files

(['train.01.jsonl',
  'train.05.jsonl',
  'train.03.jsonl',
  'train.04.jsonl',
  'train.02.jsonl'],
 ['dev.01.jsonl',
  'dev.05.jsonl',
  'dev.04.jsonl',
  'dev.03.jsonl',
  'dev.02.jsonl'],
 ['test.05.jsonl',
  'test.04.jsonl',
  'test.02.jsonl',
  'test.03.jsonl',
  'test.01.jsonl'])

In [5]:
train_files = ['train.01.jsonl']
test_files = ['test.01.jsonl']
dev_files = ['dev.01.jsonl']

In [15]:
def load_file_to_json_list(filename):
    file = os.path.join(DATASET_ROOT, filename)
    data = []
    with open(file, 'r') as f:
        json_list = list(f)
        for json_str in tqdm(json_list, desc=f'Loading data {filename}'):
            d = json.loads(json_str)
            data.append(d)
    return data

def label_to_dict_str(label_list):
    label_dict = {} # key = paragraph_id : value = label list 
    for i, label in enumerate(label_list[:]):
        label_dict[i] = label

    json_str = json.dumps(label_dict)
    num = len(label_dict)
    return json_str, num

def paragraph_to_dict_str(paragraph_list):
    paragraph_dict = {} # key = paragraph_id : value = paragraph list 
    for i, paragraph in enumerate(paragraph_list):
        new_paragraph = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            new_paragraph.append(sentence)
        paragraph_dict[i] = new_paragraph

    json_str = json.dumps(paragraph_dict)
    num = len(paragraph_dict)
    return json_str, num
def paragraph_to_text(raw_paragraph_list):
    new_paragraph_list = []
    for i, paragraph in enumerate(raw_paragraph_list):
        paragraph_list = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            paragraph_list.append(sentence)
        
        new_paragraph = ' '.join(paragraph_list)
        new_paragraph_list.append(new_paragraph)

    paragraph_str = ' '.join(new_paragraph_list)
    return paragraph_str
def summary_to_dict_str(summary_list):
    summary_dict = {} # key = summary_id : value = summary sentence 
    for i, summary in enumerate(summary_list):
        summary_dict[i] = ' '.join(summary)

    json_str = json.dumps(summary_dict)
    num = len(summary_dict)
    return json_str, num
def summary_to_text(raw_summary_list):
    summary_list = []
    for i, summary in enumerate(raw_summary_list):
        summary_list.append(' '.join(summary))

    summary_str = ' '.join(summary_list)
    return summary_str
def alter_json_data(json_list_data, filename=''):
    new_json_list = []
    for json_data in tqdm(json_list_data, desc=f'Altering json data {filename}'):
        json_data = json_data.copy()
        json_data['gold_labels'], _ = label_to_dict_str(json_data['gold_labels'])
        json_data['news_text'] = paragraph_to_text(json_data['paragraphs'])
        json_data['paragraphs'], num_paragraph = paragraph_to_dict_str(json_data['paragraphs'])
        json_data['num_of_paragraphs'] = num_paragraph
        json_data['summary_text'] = summary_to_text(json_data['summary'])
        json_data['summary'], num_summary = summary_to_dict_str(json_data['summary'])
        json_data['num_of_summary'] = num_summary
        
        new_json_list.append(json_data)
    
    return new_json_list
def create_dataset(jsonl):
    header = list(jsonl[0].keys())
    dataset_list = []
    for json_data in jsonl:
        row = []
        for h in header:
            row.append(json_data[h])
        dataset_list.append(row)
    
    return header, dataset_list
def create_dataset_from_files(file_list):
    df_header = None
    dataset_list = []
    for filename in file_list:
        json_l = load_file_to_json_list(filename)
        new_json_l = alter_json_data(json_l, filename)
        header, dataset_part = create_dataset(new_json_l)
        
        if not df_header: df_header = header
        dataset_list.extend(dataset_part)
        
    df_full = pd.DataFrame().from_records(dataset_list)
    df_full = df_full.rename(columns=dict(enumerate(header)))
    return df_full
df_train = create_dataset_from_files(train_files)
df_dev = create_dataset_from_files(dev_files)
df_test = create_dataset_from_files(test_files)

Altering json data test.01.jsonl: 100%|██████████| 3762/3762 [00:01<00:00, 2121.47it/s]


In [16]:
from datasets import Dataset, DatasetDict

# Konversi DataFrame ke Dataset
test_dataset = Dataset.from_pandas(df_test[['category', 'news_text', 'summary_text']])

In [7]:
"""from datasets import Dataset, DatasetDict

# Konversi DataFrame ke Dataset
train_dataset = Dataset.from_pandas(df_train[['news_text', 'summary_text']])
dev_dataset = Dataset.from_pandas(df_dev[['news_text', 'summary_text']])
test_dataset = Dataset.from_pandas(df_test[['news_text', 'summary_text']])

# Gabungkan menjadi DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})

# Cek struktur dataset
print(dataset)
"""

DatasetDict({
    train: Dataset({
        features: ['news_text', 'summary_text'],
        num_rows: 14262
    })
    validation: Dataset({
        features: ['news_text', 'summary_text'],
        num_rows: 750
    })
    test: Dataset({
        features: ['news_text', 'summary_text'],
        num_rows: 3762
    })
})


In [8]:
"""df_train_h = df_train[df_train['category'] == 'hiburan']
df_dev_h = df_dev[df_dev['category'] == 'hiburan']
df_test_h = df_test[df_test['category'] == 'hiburan']

train_dataset = Dataset.from_pandas(df_train_h[['news_text', 'summary_text']])
dev_dataset = Dataset.from_pandas(df_dev_h[['news_text', 'summary_text']])
test_dataset = Dataset.from_pandas(df_test_h[['news_text', 'summary_text']])
"""

In [8]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-bnb-4bit",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-07 09:15:56 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [25]:
test_dataset[2]['news_text']

'Jakarta, CNN Indonesia – Meski sudah hampir 12 tahun berlalu, film Mean Girls merupakan salah satu film remaja sepanjang masa yang sulit untuk dilupakan. Tak heran kemudian, rumah produksi film yang dibintangi Lindsay Lohan, Rachel McAdams dan Amanda Seyfried ini kemudian memutuskan akan mengadaptasinya untuk dibuat dalam versi musikal. Konfirmasi akan kepastian adaptasi dari film yang naskah skenarionya ditulis Tina Fey dan disutradara Mark Waters ini diumumkan lewat akun resmi Twitter dan Facebook MeanGirlsDC. Dijadwalkan pementasan musikal dari adaptasi naskah itu akan berlangsung September, tahun depan di Washington DC. Kabar adaptasi ke musikal ini, seperti dikutip dari NME, muncul pada Senin (3 / 10), yang juga ditandai sebagai Mean Girls Day oleh para penggemarnya. Sebelumnya, keinginan untuk mengadaptasi Mean Girls ke musikal sudah muncul pada Maret lalu. Fey pada Metro mengatakan,"Saya, suami saya, komposer Jeff Richmond, dan penulis lirik Nell Benjamin, akan bekerja sama unt

In [26]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)


In [27]:
def summarize_batch(batch):
    # Buat list isi prompt
    prompts = []
    for text in batch["news_text"]:
        messages = [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": f"Ringkaskan teks berikut:\n\n{text}",
            }]
        }]
        chat_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        prompts.append(chat_input)

    # Tokenisasi batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate semua
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.0,
            top_p=0.95,
            top_k=64
        )

    # Decode semua
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return {"model_base_generated": results}


In [105]:
"""import gc
# selesai generate
  # variabel hasil generate
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
"""

In [53]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(used_memory)

14.58


In [110]:
!nvidia-smi

Sun Apr  6 16:37:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             31W /   70W |    1103MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
df_test_h = df_test[df_test['category'] == 'hiburan']
test_dataset_h = Dataset.from_pandas(df_test_h[['category', 'news_text', 'summary_text']])

In [29]:
test_dataset_h = test_dataset_h.map(
    summarize_batch,
    batched=True,
    batch_size=64  # atau 8, 16, tergantung VRAM kamu
)


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [30]:
df = test_dataset_h.to_pandas()

In [33]:
import re

def extract_only_summary(text):
    """
    Menghapus bagian dari 'user' sampai 'model' (termasuk keduanya) dan hanya mengembalikan output model.
    Cocok jika seluruh isi (user prompt + model output) disimpan dalam satu string.
    """
    # Pola regex untuk menghapus bagian dari 'user' hingga 'model'
    pattern = re.compile(r"user.*?model\s*", re.DOTALL | re.IGNORECASE)
    cleaned_text = re.sub(pattern, "", text).strip()
    return cleaned_text


In [34]:
df["model_base_generated"] = df["model_base_generated"].apply(extract_only_summary)


In [36]:
df["model_base_generated"][351]

'.\n\n************************************************************************************************************************************************************************************************************************************************************'

In [31]:
df.head()

,category,news_text,summary_text,__index_level_0__,model_base_generated
0,hiburan,"Jakarta, CNN Indonesia – Dilansir AFP, seorang...",Eman Ahmed Abd El Aty memiliki berat badan men...,0,"user\nRingkaskan teks berikut:\n\nJakarta, CNN..."
1,hiburan,"Jakarta, CNN Indonesia – Sebuah lagu misterius...",Sebuah lagu misterius mendadak muncul di iTune...,5,"user\nRingkaskan teks berikut:\n\nJakarta, CNN..."
2,hiburan,Teh sudah jadi bagian yang tak terpisahkan dar...,Teh memiliki fungsi istimewa yang ampuh mengur...,14,user\nRingkaskan teks berikut:\n\nTeh sudah ja...
3,hiburan,"Seorang ibu di Massachusetts, Amerika Serikat,...","Seorang ibu di Massachusetts, AS, memenangkan ...",15,user\nRingkaskan teks berikut:\n\nSeorang ibu ...
4,hiburan,Butuh waktu bertahun - tahu lamanya bagi seseo...,Ketekunan dan kegigihan akan menghasilkan kual...,36,user\nRingkaskan teks berikut:\n\nButuh waktu ...


In [150]:
import gc
# selesai generate
  # variabel hasil generate
del summarize_batch
gc.collect()
torch.cuda.empty_cache()


In [151]:
!nvidia-smi

Sun Apr  6 17:12:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             32W /   70W |    1931MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [134]:
from unsloth import FastModel
from peft import PeftModel

# 2. Pasang adapter hasil fine-tune
adapter_path = "/kaggle/input/adapter/kaggle/working/results/checkpoint-255/"  # ganti sesuai lokasi adapter kamu
model = PeftModel.from_pretrained(model, adapter_path)


In [152]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
def summarize_batch(batch):
    # Buat list isi prompt
    prompts = []
    for text in batch["news_text"]:
        messages = [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": f"Ringkaskan teks berikut:\n\n{text}",
            }]
        }]
        chat_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        prompts.append(chat_input)

    # Tokenisasi batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate semua
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.0,
            top_p=0.95,
            top_k=64
        )

    # Decode semua
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return {"model_finetune_generated": results}


In [153]:
test_dataset_h = test_dataset_h.map(
    summarize_batch,
    batched=True,
    batch_size=32  # atau 8, 16, tergantung VRAM kamu
)


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [154]:
test_dataset_h_result = test_dataset_h.to_pandas()

In [163]:
print(test_dataset_h_result.iloc[2]['model_finetune_generated'])

user
Ringkaskan teks berikut:

Teh sudah jadi bagian yang tak terpisahkan dari kehidupan orang Indonesia . Jika tak percaya , coba perhatikan kembali minuman yang dipesan banyak orang setiap kali kamu makan di sebuah restoran . Jawaban yang terdengar pasti tak akan jauh dari yang namanya ' es teh manis ' , ' es teh tawar ' , atau ' teh manis hangat ' . Apakah kamu termasuk salah satunya ? Sejak dulu , teh memang dikenal sebagai minuman yang memiliki khasiat baik bagi kesehatan . Semua karena kandungan antioksidan , kafein , polifenol , dan vitaminnya yang beragam . Minuman ini juga ampuh untuk mengenyahkan racun dari dalam tubuh , lho . Jika rajin minum teh , kamu bisa terhindar dari sakit kepala dan kolesterol . Jantung dan pencernaan juga jadi lebih sehat . Dan hebatnya lagi , teh terbukti mampu memberi efek menenangkan secara psikologis bagi peminumnya . Masih meragukan khasiat baik teh bagi kesehatan manusia ? Khusus untuk kaum hawa pecinta teh , ada kabar gembira untuk kamu ! Kare

In [167]:
print(test_dataset_h_result.iloc[5]['summary_text'])


Meditasi juga mampu membawa banyak manfaat menguntungkan untuk kesehatan fisik anda . Meditasi secara tidak langsung dapat berdampak pula pada penurunan tekanan darah anda . Banyak penelitian yang telah membuktikan bahwa meditasi juga mampu meningkatkan kesehatan pencernaan , dapat memperbaiki pola tidur , dapat meningkatkan kesuburan , dan juga dengan meditasi secara teratur maka dapat meningkatkan sistem kesehatan internal tubuh .


In [197]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : """Ringkaskan teks berikut:\n\n 'Jakarta , CNN Indonesia - - Sebuah lagu misterius mendadak muncul di iTunes akhir pekan lalu . Lagu itu berjudul Behind Me , disebut sebagai karya dari seorang artis tak terkenal bernama Guido Dos Santos . Lagu itu diperkenalkan ke iTunes tanpa keterangan apa pun , lalu dihapus . Namun Gay Times melaporkan , lagu itu sempat membuat tertarik banyak orang . Bahkan ia masuk daftar 50 besar lagu laris didengar di delapan negara , meski tak disebutkan mana saja . Mengutip Independent , lagu itu dipercaya merupakan versi demo dari karya Madonna yang berjudul Two Steps Behind Me . Itu merupakan demo yang direkam Madonna pada 2015 untuk album Rebel Heart . Saat itu Two Steps Behind Me disebut berkaitan dengan Lady Gaga . Namun manajer Madonna , Guy Oseary menolak gagasan itu . “ Lagu itu bukan tentang Gaga atau siapa pun , ” tulisnya di Twitter pada 2014 . Oseary maupun Madonna tidak berkata apa-apa tentang lagu pendek Behind Me yang baru muncul di iTunes . Alih - alih , ia justru merespons perbincangan soal biopiknya , Blond Ambition . Ia tidak merestui film yang dibikin tanpa persetujuannya itu . Madonna menulis , “ Tidak seorang pun tahu apa yang saya ketahui dan yang saya lihat . Hanya saya yang bisa mengisahkan cerita hidup saya . Orang lain yang mencobanya hanya terlihat mencari keuntungan dan bodoh , mencari gratifikasi tanpa bekerja apa-apa , ” tulisnya . Tidak ada pemberitaan lebih lanjut bagaimana akhirnya nasib film itu .'""",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

["<bos><bos><start_of_turn>user\nRingkaskan teks berikut:\n\n 'Jakarta , CNN Indonesia - - Sebuah lagu misterius mendadak muncul di iTunes akhir pekan lalu . Lagu itu berjudul Behind Me , disebut sebagai karya dari seorang artis tak terkenal bernama Guido Dos Santos . Lagu itu diperkenalkan ke iTunes tanpa keterangan apa pun , lalu dihapus . Namun Gay Times melaporkan , lagu itu sempat membuat tertarik banyak orang . Bahkan ia masuk daftar 50 besar lagu laris didengar di delapan negara , meski tak disebutkan mana saja . Mengutip Independent , lagu itu dipercaya merupakan versi demo dari karya Madonna yang berjudul Two Steps Behind Me . Itu merupakan demo yang direkam Madonna pada 2015 untuk album Rebel Heart . Saat itu Two Steps Behind Me disebut berkaitan dengan Lady Gaga . Namun manajer Madonna , Guy Oseary menolak gagasan itu . “ Lagu itu bukan tentang Gaga atau siapa pun , ” tulisnya di Twitter pada 2014 . Oseary maupun Madonna tidak berkata apa-apa tentang lagu pendek Behind Me ya

In [168]:
import gc
# selesai generate
  # variabel hasil generate
del summarize_batch, model, tokenizer
gc.collect()
torch.cuda.empty_cache()


In [169]:
!nvidia-smi

Sun Apr  6 17:29:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             31W /   70W |    1963MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [172]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)


def summarize_batch(batch):
    # Buat list isi prompt
    prompts = []
    for text in batch["news_text"]:
        messages = [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": f"""
                Bacalah teks berita berikut ini.
Tugas Anda adalah menuliskan ringkasan dari teks tersebut secara padat dan langsung ke inti informasi, menggunakan gaya bahasa berita harian, tanpa tambahan pembuka seperti Ringkasan: atau Kesimpulan:, dan hanya terdiri dari dua sampai lima kalimat pendek.
\n\n{text}""",
            }]
        }]
        chat_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        prompts.append(chat_input)

    # Tokenisasi batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate semua
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.0,
            top_p=0.95,
            top_k=64
        )

    # Decode semua
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return {"model_base_zeroshot_generated": results}


In [173]:
test_dataset_h = test_dataset_h.map(
    summarize_batch,
    batched=True,
    batch_size=32  # atau 8, 16, tergantung VRAM kamu
)


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [174]:
test_dataset_h_result = test_dataset_h.to_pandas()

In [202]:
import gc
# selesai generate
  # variabel hasil generate
del results
gc.collect()
torch.cuda.empty_cache()


NameError: name 'results' is not defined

In [203]:
!nvidia-smi

Sun Apr  6 18:03:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             31W /   70W |    2795MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [179]:
# 2. Pasang adapter hasil fine-tune
adapter_path = "/kaggle/input/adapter172/kaggle/working/results/checkpoint-172/"  # ganti sesuai lokasi adapter kamu
model = PeftModel.from_pretrained(model, adapter_path)


In [204]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
def summarize_batch(batch):
    # Buat list isi prompt
    prompts = []
    for text in batch["news_text"]:
        messages = [{
            "role": "user",
            "content": [{
                "type": "text",
                "text": f"Ringkaskan teks berikut:\n\n{text}",
            }]
        }]
        chat_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        prompts.append(chat_input)

    # Tokenisasi batch
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate semua
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.0,
            top_p=0.95,
            top_k=64
        )

    # Decode semua
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return {"model_finetune172_generated": results}


In [205]:
test_dataset_h = test_dataset_h.map(
    summarize_batch,
    batched=True,
    batch_size=8  # atau 8, 16, tergantung VRAM kamu
)


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [206]:
test_dataset_h_result = test_dataset_h.to_pandas()


In [208]:
test_dataset_h_result['model_finetune172_generated'][1]

'user\nRingkaskan teks berikut:\n\nJakarta , CNN Indonesia - - Sebuah lagu misterius mendadak muncul di iTunes akhir pekan lalu . Lagu itu berjudul Behind Me , disebut sebagai karya dari seorang artis tak terkenal bernama Guido Dos Santos . Lagu itu diperkenalkan ke iTunes tanpa keterangan apa pun , lalu dihapus . Namun Gay Times melaporkan , lagu itu sempat membuat tertarik banyak orang . Bahkan ia masuk daftar 50 besar lagu laris didengar di delapan negara , meski tak disebutkan mana saja . Mengutip Independent , lagu itu dipercaya merupakan versi demo dari karya Madonna yang berjudul Two Steps Behind Me . Itu merupakan demo yang direkam Madonna pada 2015 untuk album Rebel Heart . Saat itu Two Steps Behind Me disebut berkaitan dengan Lady Gaga . Namun manajer Madonna , Guy Oseary menolak gagasan itu . “ Lagu itu bukan tentang Gaga atau siapa pun , ” tulisnya di Twitter pada 2014 . Oseary maupun Madonna tidak berkata apa-apa tentang lagu pendek Behind Me yang baru muncul di iTunes . A

In [187]:
test_dataset_h['summary_text'][0]

'Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Kini , berat badannya telah turun drastis sebanyak 323 kilogram dalam tiga bulan . Sekarang berat badannya tinggal 176,6 kilogram .'

In [209]:
test_dataset_h_result.to_csv('data_hasil_inference')